# ASP Emulator
## Problem Instance

In [66]:
%%file instance-2.lp

#const maxtime = 6.

apicall(1, get_something).
cmp_imm(2, eax, 0).
jne(3, 5).
eicar(4).
end(5).

Overwriting instance-2.lp


In [75]:
%%file instance-3.lp

#const maxtime = 3.

apicall(1, get_something).
eicar(2).
end(3).

Writing instance-3.lp


## Solution

In [67]:
%%file solution-2.lp

apicall_res(1, 0).
malware_sig_eicar(4).

Overwriting solution-2.lp


## Sample Formulation

In [72]:
%%file sample-formulation.lp

#const intlen = 32.

register(eax;ebx;ecx;edx;esi;edi;edp;esp;eip).
flag(n;z;p).
pos(0..intlen-1).
bit(0;1).
imm8(-128..127).
time(T) :- T = 0..maxtime.
regbit(0, Reg, Pos, 0) :- register(Reg), pos(Pos).
flg(0, Flag, 0) :- flag(Flag).
    
:- regbit(T, Reg, Pos, B1), regbit(T, Reg, Pos, B2), B1 > B2, register(Reg), pos(Pos), bit(B1), bit(B2), time(T).
:- flg(T, Flag, B1), flg(T, Flag, B2), B1 > B2, flag(Flag), bit(B1), bit(B2), time(T).
    
cmp(PC) :- cmp_imm(PC, Reg, Val), register(Reg), imm8(Val).
%cmp(PC) :- cmp_reg(PC, Reg1, Reg2), register(Reg1), register(Reg2).
    
flg(T, Flag, Val) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    flg(T-1, Flag, Val),
    not cmp(PC).

flg(T, n, 0) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal >= Val.
    
flg(T, n, 1) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal < Val.
    
flg(T, z, 0) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal != Val.
    
flg(T, z, 1) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal = Val.
    
flg(T, p, 0) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal <= Val.
    
flg(T, p, 1) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal > Val.

regbit(T, Reg, Pos, Val) :-
    time(T),
    register(Reg),
    pos(Pos),
    Reg != eip,
    regval(T, eip, PC),
    regbit(T-1, Reg, Pos, Val),
    Reg != eax.
    
regbit(T, eax, Pos, Val) :-
    time(T),
    pos(Pos),
    regval(T, eip, PC),
    regbit(T-1, eax, Pos, Val),
    not apicall(PC, _).
    
regbit(T, eip, Pos, B) :-
    time(T),
    pos(Pos),
    bit(B),
    Pos > 0,
    regbit(T-1, eip, Pos, B),
    regbit(T-1, eip, Pos-1, 0),
    regval(T-1, eip, PC),
    %not jmp(PC, _),
    %not je(PC, _),
    not jne(PC, _),
    %not jg(PC, _),
    %not jge(PC, _),
    %not jl(PC, _),
    %not jle(PC, _),
    not end(PC).
regbit(T, eip, Pos, B) :-
    time(T),
    pos(Pos),
    bit(B),
    Pos > 0,
    regbit(T-1, eip, Pos, B),
    regbit(T-1, eip, Pos-1, 1),
    regbit(T, eip, Pos-1, 1),
    regval(T-1, eip, PC),
    %not jmp(PC, _),
    %not je(PC, _),
    not jne(PC, _),
    %not jg(PC, _),
    %not jge(PC, _),
    %not jl(PC, _),
    %not jle(PC, _),
    not end(PC).
regbit(T, eip, Pos, B) :-
    time(T),
    pos(Pos),
    bit(B),
    Pos > 0,
    regbit(T-1, eip, Pos, B^1),
    regbit(T-1, eip, Pos-1, 1),
    regbit(T, eip, Pos-1, 0),
    regval(T-1, eip, PC),
    %not jmp(PC, _),
    %not je(PC, _),
    not jne(PC, _),
    %not jg(PC, _),
    %not jge(PC, _),
    %not jl(PC, _),
    %not jle(PC, _),
    not end(PC).
regbit(T, eip, 0, B) :-
    time(T),
    bit(B),
    regbit(T-1, eip, Pos, B^1),
    regval(T-1, eip, PC),
    %not jmp(PC, _),
    %not je(PC, _),
    not jne(PC, _),
    %not jg(PC, _),
    %not jge(PC, _),
    %not jl(PC, _),
    %not jle(PC, _),
    not end(PC).
    
regbit(T, eip, Pos, B) :-
    time(T),
    pos(Pos),
    jne(PC2, PC),
    regval(T-1, eip, PC2),
    PC / (2 ** Pos) & 1 = B,
    flg(T-1, z, 0).
    
regbit(T, eip, Pos, B) :-
    time(T),
    pos(Pos),
    jne(PC-1, _),
    regval(T-1, eip, PC-1),
    PC / (2 ** Pos) & 1 = B,
    flg(T-1, z, 1).

1 { apicall_res(T, Pos, B):bit(B) } 1 :-
    time(T),
    pos(Pos),
    regval(T, eip, PC),
    apicall(PC, get_something).
    
regbit(T, eax, Pos, B) :-
    time(T),
    pos(Pos),
    apicall_res(T, Pos, B),
    regval(T, eip, PC),
    apicall(PC, _).
    
malware_sig_eicar(T) :-
    time(T),
    regval(T, eip, PC),
    eicar(PC).
    
malware_sig :- malware_sig_eicar(T), time(T).
    
%:- not malware_sig.
    
%#show malware_sig_eicar/1.
%#show apicall_res/3.
#show regval/3.

Overwriting sample-formulation.lp


In [73]:
%%file regval32.lp

regval(T, Reg, Val) :-
    time(T),
    register(Reg),
    regbit(T, Reg, 0, B0),
    regbit(T, Reg, 1, B1),
    regbit(T, Reg, 2, B2),
    regbit(T, Reg, 3, B3),
    regbit(T, Reg, 4, B4),
    regbit(T, Reg, 5, B5),
    regbit(T, Reg, 6, B6),
    regbit(T, Reg, 7, B7),
    regbit(T, Reg, 8, B8),
    regbit(T, Reg, 9, B9),
    regbit(T, Reg, 10, B10),
    regbit(T, Reg, 11, B11),
    regbit(T, Reg, 12, B12),
    regbit(T, Reg, 13, B13),
    regbit(T, Reg, 14, B14),
    regbit(T, Reg, 15, B15),
    regbit(T, Reg, 16, B16),
    regbit(T, Reg, 17, B17),
    regbit(T, Reg, 18, B18),
    regbit(T, Reg, 19, B19),
    regbit(T, Reg, 20, B20),
    regbit(T, Reg, 21, B21),
    regbit(T, Reg, 22, B22),
    regbit(T, Reg, 23, B23),
    regbit(T, Reg, 24, B24),
    regbit(T, Reg, 25, B25),
    regbit(T, Reg, 26, B26),
    regbit(T, Reg, 27, B27),
    regbit(T, Reg, 28, B28),
    regbit(T, Reg, 29, B29),
    regbit(T, Reg, 30, B30),
    regbit(T, Reg, 31, B31),
    Val = (B31 * (2 ** 31)) +
          (B30 * (2 ** 30)) +
          (B29 * (2 ** 29)) +
          (B28 * (2 ** 28)) +
          (B27 * (2 ** 27)) +
          (B26 * (2 ** 26)) +
          (B25 * (2 ** 25)) +
          (B24 * (2 ** 24)) +
          (B23 * (2 ** 23)) +
          (B22 * (2 ** 22)) +
          (B21 * (2 ** 21)) +
          (B20 * (2 ** 20)) +
          (B19 * (2 ** 19)) +
          (B18 * (2 ** 18)) +
          (B17 * (2 ** 17)) +
          (B16 * (2 ** 16)) +
          (B15 * (2 ** 15)) +
          (B14 * (2 ** 14)) +
          (B13 * (2 ** 13)) +
          (B12 * (2 ** 12)) +
          (B11 * (2 ** 11)) +
          (B10 * (2 ** 10)) +
          (B9 * (2 ** 9)) +
          (B8 * (2 ** 8)) +
          (B7 * (2 ** 7)) +
          (B6 * (2 ** 6)) +
          (B5 * (2 ** 5)) +
          (B4 * (2 ** 4)) +
          (B3 * (2 ** 3)) +
          (B2 * (2 ** 2)) +
          (B1 * (2 ** 1)) + B0.

Overwriting regval32.lp


In [74]:
!clingo regval32.lp instance-2.lp sample-formulation.lp --time-limit=600 | tail

clingo version 5.1.0
Reading from regval32.lp ...
UNKNOWN

TIME LIMIT   : 1
Models       : 0+
Calls        : 1
Time         : 604.816s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 166.688s


*** Info : (clingo): INTERRUPTED by signal!
